In [1]:
from b_get_data import *
from c_model_related import CustomStandardScaler as Cscl
from c_model_related import Trainer as tr
from c_model_related import metrics_create_df
import joblib

In [2]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]


feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

In [3]:
# Obter o caminho dos ficheiros.
root_dir = os.path.abspath('..')
csv_path = os.path.join(root_dir, 'rawdata')
# Importar o dataset de failures
failures_path = os.path.join(csv_path, 'wind-farm-1-failures-training.csv')
failures_df = get_data(failures_path)
# Importar o dataset de signals
signals_path = os.path.join(csv_path, 'wind-farm-1-signals-training.csv')
signals_df = get_data(signals_path)
# Cortar colunas que não têm valores
cols_to_drop = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']
signals_df = signals_df.drop(columns=cols_to_drop)
signals_df

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,T11,2016-01-01,1339.4,1233.3,1270.9,18.8,42.0,59,61,60,...,-973.4,-752.6,-1000.0,56.5,973.4,1000.0,752.6,56.5,38,206.9
1,T06,2016-01-01,1270.0,1232.8,1248.5,6.8,42.0,51,52,53,...,-307.1,0.0,-880.6,180.0,307.1,880.6,0.0,180.0,35,204.6
2,T01,2016-01-01,1277.4,1226.1,1249.0,9.0,41.0,58,59,58,...,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37,218.5
3,T09,2016-01-01,1376.7,1234.3,1272.0,24.7,40.0,58,57,58,...,-861.6,-224.5,-1000.0,185.5,861.6,1000.0,224.5,185.5,33,214.0
4,T07,2016-01-01,1317.5,1229.5,1254.9,13.8,41.0,62,62,61,...,-536.5,0.0,-1000.0,338.9,536.5,1000.0,0.0,338.9,39,197.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434140,T11,2017-09-01,1266.3,1234.3,1250.6,5.1,46.0,60,61,61,...,-864.5,-524.4,-1000.0,109.1,864.5,1000.0,524.4,109.1,43,96.8
434141,T09,2017-09-01,1268.4,1240.1,1251.9,6.0,44.0,62,61,62,...,-900.6,-340.3,-1000.0,121.6,900.6,1000.0,340.3,121.6,40,105.0
434142,T06,2017-09-01,1284.2,1233.4,1252.4,8.8,49.0,63,64,65,...,-725.4,-171.8,-1000.0,272.2,725.4,1000.0,171.8,272.2,45,84.1
434143,T01,2017-09-01,1299.0,1222.1,1255.3,12.4,45.0,64,64,64,...,-822.6,-314.2,-1000.0,203.6,822.6,1000.0,314.2,203.6,44,109.5


In [4]:
# print('002 - Criar o dicionário com os Dataframes originais')
df_dict = {'failures_df':failures_df, 'signals_df':signals_df}

In [5]:
df_dict.keys()

dict_keys(['failures_df', 'signals_df'])

In [6]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = component_df_creation(
    signals_df)

In [7]:
comp_df_dict = {
    'df_generator': df_generator,
    'df_hydraulic': df_hydraulic,
    'df_gen_bear': df_gen_bear,
    'df_transformer': df_transformer,
    'df_gearbox': df_gearbox
}

In [8]:
for key in comp_df_dict:
    print(comp_df_dict[key].shape)

(434145, 45)
(434145, 34)
(434145, 45)
(434145, 36)
(434145, 33)


In [9]:
comp_prep_df_dict = comp_df_dict.copy()

In [10]:
component_list = [
    'GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
    'GEARBOX'
]
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = sig_fail_merge_dfs(sig_df=comp_prep_df_dict[key],
                                                fail_df=failures_df,
                                                component=component_list[i])

In [11]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 46)
(434145, 35)
(434145, 46)
(434145, 37)
(434145, 34)


In [12]:
turbine_list = ['T11', 'T06', 'T01', 'T09', 'T07']
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = fill_na_by_turbine(comp_prep_df_dict[key],
                                                turbine_list)

/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:120: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['date'] = df_merged['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/wmillfailprev/b_get_data.py:121: SettingWithCopyWarning: 
A value is trying to be

In [13]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

(434147, 47)
(434145, 36)
(434145, 47)
(434145, 38)
(434145, 35)


In [14]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].isna().sum().value_counts(), key)

0    47
dtype: int64 df_generator
0    36
dtype: int64 df_hydraulic
0    47
dtype: int64 df_gen_bear
0    38
dtype: int64 df_transformer
0    35
dtype: int64 df_gearbox


In [15]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = aplic_var_target(comp_prep_df_dict[key], 60)

In [16]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 48) df_generator
(434145, 37) df_hydraulic
(434145, 48) df_gen_bear
(434145, 39) df_transformer
(434145, 36) df_gearbox


In [17]:
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = comp_prep_df_dict[key].drop(
        columns=feat_drop_list[i])

In [18]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(434147, 31) df_generator
(434145, 26) df_hydraulic
(434145, 31) df_gen_bear
(434145, 28) df_transformer
(434145, 26) df_gearbox


In [19]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = group_por_frequency(comp_prep_df_dict[key],
                                                 period='Dia')

In [20]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 31) df_generator
(3041, 19) df_hydraulic
(3041, 18) df_gen_bear
(3041, 19) df_transformer
(3041, 16) df_gearbox


In [21]:
cols = ['Turbine_ID', 'Date', 'TTF', '60_days']

In [22]:
def add_features2(df_in, rolling_win_size=15):
    """Add rolling average and rolling standard deviation for sensors readings using fixed rolling window size.
    """
    cols =['Turbine_ID', 'Date', 'TTF', '60_days']
    other_cols = []
    for i in df_in.columns:
        if i not in cols:
            other_cols.append(i)
    all_cols = cols + other_cols
    
    df_in = df_in[all_cols]
    
    sensor_cols = []
    for i in df_in.columns[4:]:
        sensor_cols.append(i)

    sensor_av_cols = [nm+'_av' for nm in sensor_cols]
    sensor_sd_cols = [nm+'_sd' for nm in sensor_cols]

    df_out = pd.DataFrame()

    ws = rolling_win_size

    #calculate rolling stats for each engine id

    for m_id in pd.unique(df_in.Turbine_ID):

        # get a subset for each engine sensors
        df_engine = df_in[df_in['Turbine_ID'] == m_id]
        df_sub = df_engine[sensor_cols]

        # get rolling mean for the subset
        av = df_sub.rolling(ws, min_periods=1).mean()
        av.columns = sensor_av_cols

        # get the rolling standard deviation for the subset
        sd = df_sub.rolling(ws, min_periods=1).std().fillna(0)
        sd.columns = sensor_sd_cols

        # combine the two new subset dataframes columns to the engine subset
        new_ftrs = pd.concat([df_engine,av,sd], axis=1)

        # add the new features rows to the output dataframe
        df_out = pd.concat([df_out,new_ftrs])
    df_out = df_out.sort_values(by=['Turbine_ID', 'Date']   )
    return df_out

In [23]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = add_features2(comp_prep_df_dict[key], rolling_win_size=10)

In [24]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

(3042, 85) df_generator
(3041, 49) df_hydraulic
(3041, 46) df_gen_bear
(3041, 49) df_transformer
(3041, 40) df_gearbox


In [65]:
df_train_comp_dict = {}
df_test_comp_dict = {}
for key in comp_prep_df_dict:
    df_train_comp_dict[key] = prepare_train_df(comp_prep_df_dict[key], meses = 3)
    df_test_comp_dict[key] = prepare_test_df(comp_prep_df_dict[key], meses = 3)

In [66]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


In [67]:
for key in df_train_comp_dict:
    print(df_train_comp_dict[key].shape, key)
for key in df_test_comp_dict:
    print(df_test_comp_dict[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox


In [68]:
print('012 - separar entre x_train, x_test')
x_train = df_train_comp_dict.copy()
y_train = df_train_comp_dict.copy()
x_test = df_test_comp_dict.copy()
y_test = df_test_comp_dict.copy()

012 - separar entre x_train, x_test


In [69]:
print('xxx - transformar o y de float para int')
col_to_mantain_test = ['60_days']
for key in y_train:
    y_train[key] = y_train[key][col_to_mantain_test].to_numpy().astype(int)[:,0]
    y_test[key] = y_test[key][col_to_mantain_test].to_numpy().astype(int)[:,0]

xxx - transformar o y de float para int


In [70]:
for key in x_train:
    print(x_train[key].shape, key)
for key in y_train:
    print(y_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)
for key in y_test:
    print(y_test[key].shape, key)

(2577, 85) df_generator
(2576, 49) df_hydraulic
(2576, 46) df_gen_bear
(2576, 49) df_transformer
(2576, 40) df_gearbox
(2577,) df_generator
(2576,) df_hydraulic
(2576,) df_gen_bear
(2576,) df_transformer
(2576,) df_gearbox
(465, 85) df_generator
(465, 49) df_hydraulic
(465, 46) df_gen_bear
(465, 49) df_transformer
(465, 40) df_gearbox
(465,) df_generator
(465,) df_hydraulic
(465,) df_gen_bear
(465,) df_transformer
(465,) df_gearbox


In [71]:
print('012 - separar entre x_train, x_test')
# retirar as colunas que não devem entrar no x
cols_to_drop_train = ['Date', 'TTF', '60_days','Component','Component_av','Component_sd']
for key in x_train:
    x_train[key] = x_train[key].drop(columns=cols_to_drop_train)
    x_test[key] = x_test[key].drop(columns=cols_to_drop_train)

012 - separar entre x_train, x_test


In [72]:
for key in x_train:
    print(x_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)

(2577, 79) df_generator
(2576, 43) df_hydraulic
(2576, 40) df_gen_bear
(2576, 43) df_transformer
(2576, 34) df_gearbox
(465, 79) df_generator
(465, 43) df_hydraulic
(465, 40) df_gen_bear
(465, 43) df_transformer
(465, 34) df_gearbox


In [73]:
x_train['df_generator'][x_train['df_generator']['Turbine_ID']=='T01']['Blds_PitchAngle_Std'].std()

0.801278546982223

In [74]:
print('014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro')
print('014.1 - df_generator')
scaler_df_generator = Cscl()
scaler_df_generator.fit(x_train['df_generator'])
# joblib.dump(scaler_df_generator, 'scaler_df_generator.joblib')
print('014.2 - df_hydraulic')
scaler_df_hydraulic = Cscl()
scaler_df_hydraulic.fit(x_train['df_hydraulic'])
# joblib.dump(scaler_df_hydraulic, 'scaler_df_hydraulic.joblib')
print('014.3 - df_gen_bear')
scaler_df_gen_bear = Cscl()
scaler_df_gen_bear.fit(x_train['df_gen_bear'])
# joblib.dump(scaler_df_gen_bear, 'scaler_df_gen_bear.joblib')
print('014.4 - df_transformer')
scaler_df_transformer = Cscl()
scaler_df_transformer.fit(x_train['df_transformer'])
# joblib.dump(scaler_df_transformer, 'scaler_df_transformer.joblib')
print('014.5 - df_gearbox')
scaler_df_gearbox = Cscl()
scaler_df_gearbox.fit(x_train['df_gearbox'])
# joblib.dump(scaler_df_gearbox, 'scaler_df_gearbox.joblib')

014 - fazer o fit do CustomTransfMiguel para cada componente à pedreiro
014.1 - df_generator
014.2 - df_hydraulic
014.3 - df_gen_bear
014.4 - df_transformer
014.5 - df_gearbox


CustomStandardScaler()

x_train['df_generator'].columns[9]

x_train['df_generator']['Blds_PitchAngle_Std'].isna().sum()

In [75]:
x_train['df_generator'].columns

Index(['Turbine_ID', 'Gen_RPM_Avg', 'Gen_RPM_Std', 'Gen_Bear_Temp_Avg', 'Gen_Phase2_Temp_Avg', 'Prod_LatestAvg_ActPwrGen0', 'Prod_LatestAvg_ActPwrGen1', 'Prod_LatestAvg_ReactPwrGen0',
       'Gen_SlipRing_Temp_Avg', 'Gen_Bear2_Temp_Avg', 'Grd_RtrInvPhase2_Temp_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min', 'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Std', 'Amb_WindDir_Relative_Avg',
       'Amb_WindDir_Abs_Avg', 'Amb_Temp_Avg', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std', 'Cont_Top_Temp_Avg', 'Cont_VCP_ChokcoilTemp_Avg', 'Cont_VCP_WtrTemp_Avg', 'Nac_Temp_Avg',
       'Nac_Direction_Avg', 'Grd_Busbar_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Gen_RPM_Avg_av', 'Gen_RPM_Std_av', 'Gen_Bear_Temp_Avg_av', 'Gen_Phase2_Temp_Avg_av', 'Prod_LatestAvg_ActPwrGen0_av',
       'Prod_LatestAvg_ActPwrGen1_av', 'Prod_LatestAvg_ReactPwrGen0_av', 'Gen_SlipRing_Temp_Avg_av', 'Gen_Bear2_Temp_Avg_av', 'Grd_RtrInvPhase2_Temp_Avg_av', 'Amb_WindSpeed_Max_av',
       'Amb_WindSpeed_Min_av', 'Amb_WindSpeed_Avg_av', 'Amb_WindSpee

In [77]:
x_train['df_generator'][x_train['df_generator']['Turbine_ID']=='T11'].tail()

,Turbine_ID,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase2_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ReactPwrGen0,Gen_SlipRing_Temp_Avg,Gen_Bear2_Temp_Avg,...,Amb_Temp_Avg_sd,Blds_PitchAngle_Avg_sd,Blds_PitchAngle_Std_sd,Cont_Top_Temp_Avg_sd,Cont_VCP_ChokcoilTemp_Avg_sd,Cont_VCP_WtrTemp_Avg_sd,Nac_Temp_Avg_sd,Nac_Direction_Avg_sd,Grd_Busbar_Temp_Avg_sd,Hyd_Oil_Temp_Avg_sd
2944,T11,736.902778,75.672222,41.583333,46.652778,-429.750000,38042.041667,-397.395833,28.347222,35.430556,...,0.792944,5.576829,0.855285,4.448392,24.787123,3.341270,1.929151,49.152803,6.565248,6.511922
2945,T11,1074.609028,78.825694,48.145833,55.062500,-366.583333,36732.888889,-358.569444,28.638889,37.777778,...,0.757087,5.474136,0.780862,4.473815,24.921303,3.382057,1.935353,41.116348,6.597261,6.420388
2946,T11,1055.833333,73.314583,42.270833,55.222222,-326.173611,26352.833333,-340.187500,27.555556,36.750000,...,0.882564,5.503111,0.779894,4.472685,24.674543,3.341645,2.017797,53.292026,6.558152,6.591841
2947,T11,856.822917,77.147917,41.243056,51.465278,-574.263889,19875.548611,-655.173611,27.930556,36.020833,...,0.868627,5.374963,0.778958,4.444650,23.762752,3.260856,2.106135,59.387293,6.342728,6.763157
2948,T11,1520.355556,53.554167,66.020833,82.451389,-55.048611,174227.430556,-80.236111,36.743056,53.708333,...,0.886096,5.712690,0.687436,3.687723,20.784595,2.920003,1.776151,60.896827,5.065998,5.285825


In [78]:
scaler_df_generator.mean_['T11']

array([ 1.12805845e+03,  6.93712130e+01,  4.73977556e+01,  6.31287677e+01,
       -2.91951271e+02,  9.50649182e+04, -1.80151076e+02,  2.94872432e+01,
        4.16225137e+01,  3.99359172e+01,  1.34219658e+01,  1.52780732e+00,
        6.06483790e+00,  1.23993700e+00,  3.86405758e-01,  1.82192790e+02,
        1.90459900e+01,  6.55460444e+00,  1.74658945e+00,  3.98308083e+01,
        7.84958025e+01,  3.73283006e+01,  2.80134676e+01,  1.87042206e+02,
        3.77941695e+01,  3.34715502e+01,  1.12820458e+03,  6.92545188e+01,
        4.73405656e+01,  6.31071943e+01, -2.91175832e+02,  9.52027971e+04,
       -1.78490375e+02,  2.94483226e+01,  4.15973000e+01,  3.99267679e+01,
        1.34264096e+01,  1.52623315e+00,  6.06637633e+00,  1.24043875e+00,
        4.27455524e-01,  1.82606954e+02,  1.90033654e+01,  6.55032205e+00,
        1.74101968e+00,  3.98131660e+01,  7.84895576e+01,  3.73139654e+01,
        2.79853129e+01,  1.87503793e+02,  3.77729608e+01,  3.34307963e+01,
        3.60896310e+02,  

In [79]:
scaler_df_generator.std_['T11']

array([4.01775838e+02, 2.33755017e+01, 1.21939377e+01, 1.91487855e+01,
       2.51051818e+02, 8.77971916e+04, 1.68043124e+02, 7.03443114e+00,
       8.84986339e+00, 4.67712969e+00, 5.23330304e+00, 6.54551795e-01,
       2.71914260e+00, 5.76405409e-01, 6.27907823e+00, 7.12822465e+01,
       4.72323694e+00, 7.82956438e+00, 8.32997634e-01, 4.43259897e+00,
       1.99458744e+01, 3.86203940e+00, 4.18697985e+00, 7.66568889e+01,
       5.65793940e+00, 6.65616600e+00, 1.95256333e+02, 9.01351199e+00,
       6.85250115e+00, 1.04096299e+01, 1.15365394e+02, 4.41412390e+04,
       7.35881420e+01, 5.43384557e+00, 5.37603504e+00, 3.02592763e+00,
       2.54492285e+00, 3.65169377e-01, 1.37828194e+00, 2.84621734e-01,
       2.13863368e+00, 3.97844522e+01, 4.42549590e+00, 3.48849596e+00,
       3.74076742e-01, 3.42025409e+00, 1.03964614e+01, 2.88453649e+00,
       3.76061315e+00, 4.25935969e+01, 3.96320858e+00, 5.02042883e+00,
       8.41074420e+01, 6.95981068e+00, 2.93860329e+00, 5.16154616e+00,
      

In [45]:
print('012.1 - considerar no y set apenas as turbinas que tiveram falhas')
train_turbines = {
    'df_generator': ['T11', 'T06'],
    'df_hydraulic': ['T06', 'T11'],
    'df_gen_bear': ['T07', 'T09'],
    'df_transformer': ['T07'],
    'df_gearbox': ['T09']
}
for key in df_train_comp_dict:
    df_train_comp_dict[key] = df_train_comp_dict[key][
        df_train_comp_dict[key]['Turbine_ID'].isin(train_turbines[key])]

012.1 - considerar no y set apenas as turbinas que tiveram falhas


x_train1 = {}
x_test1 = {}

def fit2(X, y=None):
    mean_ = {}
    std_ = {}

    for turbine in ['T01','T06','T07','T09','T11']:
        X_filter = X[X['Turbine_ID']==turbine]
        X_filter = X_filter.drop(columns='Turbine_ID').to_numpy()
        mean_[turbine] = X_filter.mean(axis=0)
        std_[turbine] = X_filter.std(axis=0)

    return mean_,std_

def transform2(X, y=None):
    X_turbine={}

    for turbine in ['T01','T06','T07','T09','T11']:
        if std_[turbine] == 0:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / 1
        else:
            X_turbine[turbine] = (X[X['Turbine_ID']==turbine].drop(columns='Turbine_ID').to_numpy() - mean_[turbine]) / std_[turbine]

    X_turbine_all=X_turbine['T01']

    for turbine in ['T06','T07','T09','T11']:

        X_turbine_all=np.concatenate([X_turbine_all,X_turbine[turbine]])

    return X_turbine_all

mean_,std_ = fit2(x_train['df_generator'])

std_['T01']

transform(x_train['df_generator'])

In [82]:
x_train['df_generator'] 

array([[ 0.16146634,  1.01910023, -0.45888491, ..., -3.39113176,
        -3.91742406, -1.97517864],
       [-1.05315435, -0.31099475, -0.87365452, ..., -0.82958484,
        -1.06877728, -1.92384519],
       [-0.4206826 , -1.009749  , -0.18873892, ..., -0.88807398,
        -1.33082158, -1.69671974],
       ...,
       [-0.17976471,  0.16869672, -0.42044846, ..., -0.52623294,
         2.14946983,  1.20195745],
       [-0.6750917 ,  0.33268607, -0.50473442, ..., -0.22520202,
         1.96077181,  1.28563547],
       [ 0.97640791, -0.67665056,  1.52724068, ..., -0.15064969,
         0.84243332,  0.56404373]])

In [80]:
print('016 - aplicar o scale com o fit efectuado no treino')
x_train['df_generator'] = scaler_df_generator.transform(x_train['df_generator'])
x_train['df_hydraulic'] = scaler_df_hydraulic.transform(x_train['df_hydraulic'])
x_train['df_gen_bear'] = scaler_df_gen_bear.transform(x_train['df_gen_bear'])
x_train['df_transformer'] = scaler_df_transformer.transform(x_train['df_transformer'])
x_train['df_gearbox'] = scaler_df_gearbox.transform(x_train['df_gearbox'])

016 - aplicar o scale com o fit efectuado no treino


In [81]:
print('016 - aplicar o scale com o fit efectuado no teste')
x_test['df_generator'] = scaler_df_generator.transform(x_test['df_generator'])
x_test['df_hydraulic'] = scaler_df_hydraulic.transform(x_test['df_hydraulic'])
x_test['df_gen_bear'] = scaler_df_gen_bear.transform(x_test['df_gen_bear'])
x_test['df_transformer'] = scaler_df_transformer.transform(x_test['df_transformer'])
x_test['df_gearbox'] = scaler_df_gearbox.transform(x_test['df_gearbox'])

016 - aplicar o scale com o fit efectuado no teste


In [48]:
for key in x_train:
    print(x_train[key].shape, key)
for key in x_test:
    print(x_test[key].shape, key)

(2577, 81) df_generator
(2576, 45) df_hydraulic
(2576, 42) df_gen_bear
(2576, 45) df_transformer
(2576, 36) df_gearbox
(465, 81) df_generator
(465, 45) df_hydraulic
(465, 42) df_gen_bear
(465, 45) df_transformer
(465, 36) df_gearbox


exp1 = pd.DataFrame(x_test['df_generator'])

x_test['df_generator'][9]

exp1[9]

In [49]:
print('017 - Instanciar e treinar o modelo correspondente')
generator_model = tr(x_train=x_train['df_generator'],
                     y_train=y_train['df_generator'],
                     x_test=x_test['df_generator'],
                     component='df_generator')
generator_model.train()
# joblib.dump(generator_model, 'generator_model.joblib')

017 - Instanciar e treinar o modelo correspondente


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print('018 - Obter a previsão para a métrica de poupança')
y_pred_generator = generator_model.predict()